# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import Conditional_VAE as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_file = config['IO_OPTION']['OUTPUT_ROOT']+'/train_{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_file, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(777)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
#dev_dir = INPUT_ROOT + "/dev_data"
#add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = ['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
OUT_SCORE_DIR = OUTPUT_ROOT + '/score'
OUT_PRED_DIR = OUTPUT_ROOT + '/pred'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(OUT_SCORE_DIR, exist_ok=True)
os.makedirs(OUT_PRED_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/CenterLossVAE/ex1_oldfeat/config.yaml'

In [7]:
torch.cuda.is_available()

True

In [8]:
ext_data = pd.read_pickle(f'{INPUT_ROOT}/valve_features.pkl')

In [9]:
pd.DataFrame(ext_data['train']['features'])

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,0.141376,0.126192,0.160170,0.039393,0.116594,0.244344,0.364260,0.179045,0.304383,0.316418,...,0.000000,0.000000,0.000000,0.0,0.011685,0.0,0.0,0.0,0.001349,0.024873
1,0.139377,0.119569,0.153211,0.039412,0.117448,0.268657,0.362977,0.172853,0.304429,0.321536,...,0.000558,0.000000,0.000185,0.0,0.019411,0.0,0.0,0.0,0.010698,0.031158
2,0.163800,0.184355,0.172159,0.027273,0.110422,0.169510,0.386510,0.206604,0.313434,0.314130,...,0.011560,0.000000,0.000000,0.0,0.038945,0.0,0.0,0.0,0.000000,0.005341
3,0.143226,0.131679,0.156749,0.038878,0.115537,0.241695,0.368068,0.185738,0.308175,0.316420,...,0.000000,0.000457,0.000000,0.0,0.005694,0.0,0.0,0.0,0.008102,0.020985
4,0.137737,0.123774,0.149359,0.042918,0.111843,0.259588,0.368717,0.180469,0.305680,0.309504,...,0.000000,0.000000,0.000000,0.0,0.018220,0.0,0.0,0.0,0.003810,0.032367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6013,0.116490,0.110039,0.161647,0.043416,0.096190,0.239978,0.371168,0.168759,0.329152,0.293912,...,0.000000,0.000000,0.000000,0.0,0.002287,0.0,0.0,0.0,0.000226,0.023715
6014,0.125852,0.120752,0.164564,0.039160,0.104617,0.234347,0.369856,0.180599,0.320280,0.302117,...,0.001283,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.001988,0.014878
6015,0.107534,0.116570,0.173803,0.039604,0.088328,0.213753,0.390930,0.170922,0.355653,0.284926,...,0.000713,0.026625,0.000000,0.0,0.009445,0.0,0.0,0.0,0.005314,0.000665
6016,0.123527,0.140547,0.164611,0.038327,0.096956,0.192954,0.382039,0.200973,0.332633,0.290760,...,0.002548,0.011003,0.000000,0.0,0.011084,0.0,0.0,0.0,0.004229,0.010059


## training

In [10]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dataloader
    ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine_type}_features.pkl')
    dataloaders_dict = modeler.make_dataloader(ext_data)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    score_out_path = OUT_SCORE_DIR + '/{}_score.csv'.format(machine_type)
    pred_out_path = OUT_PRED_DIR + '/{}_pred.csv'.format(machine_type)
    logger.info('TRAINING')
    # parameter setting
    in_features = ext_data['train']['features'].shape[1]
    mid_size = config['param']['mid_size']
    latent_size = config['param']['latent_size']
    num_classes = 6
    net = Model(in_features, mid_size, latent_size, num_classes)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['param']['num_epochs']
    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e-2, 
    #                                          max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # training
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, num_epochs, writer, model_out_path, score_out_path, pred_out_path)
    
    com.toc()

In [11]:
machine_types

['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']

In [12]:
for machine_type in machine_types:
    run(machine_type)

2021-05-28 21:50:05,812 - 00_train.py - INFO - TARGET MACHINE_TYPE: ToyCar
2021-05-28 21:50:05,813 - 00_train.py - INFO - MAKE DATA_LOADER
2021-05-28 21:50:05,855 - 00_train.py - INFO - TRAINING
2021-05-28 21:50:09,074 - pytorch_modeler.py - INFO - train
  0%|          | 0/12 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 12/12 [00:00<00:00, 12.89it/s]
2021-05-28 21:50:10,009 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 87.26it/s]
2021-05-28 21:50:10,051 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 68.44it/s]


,AUC,pAUC
Source_0,0.620000,0.592632
Source_1,0.768300,0.606316
Source_2,0.711900,0.628947
Target_0,0.640500,0.626316
Target_1,0.892800,0.850526
Target_2,0.661400,0.629474
mean,0.715817,0.655702
h_mean,0.704849,0.646018


2021-05-28 21:50:10,769 - pytorch_modeler.py - INFO - epoch:1/100, train_losses:45268.002279, val_AUC_hmean:0.704849, val_pAUC_hmean:0.646018, best_flag:True
2021-05-28 21:50:10,770 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 25.52it/s]
2021-05-28 21:50:11,242 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 78.72it/s]
2021-05-28 21:50:11,286 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 94.24it/s]
2021-05-28 21:50:11,326 - pytorch_modeler.py - INFO - epoch:2/100, train_losses:31952.599935, val_AUC_hmean:0.614653, val_pAUC_hmean:0.591539, best_flag:False
2021-05-28 21:50:11,327 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 24.74it/s]
2021-05-28 21:50:11,815 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 87.83it/s]
2021-05-28 21:50:11,854 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 78.93it/s]
2021-05-28 21:5

...

2021-05-28 21:59:03,399 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 24.14it/s]
2021-05-28 21:59:03,898 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 82.17it/s]
2021-05-28 21:59:03,941 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 72.06it/s]
2021-05-28 21:59:03,991 - pytorch_modeler.py - INFO - epoch:56/100, train_losses:9141.226562, val_AUC_hmean:0.680891, val_pAUC_hmean:0.598329, best_flag:False
2021-05-28 21:59:03,992 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 24.78it/s]
2021-05-28 21:59:04,479 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 84.64it/s]
2021-05-28 21:59:04,519 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 68.37it/s]
2021-05-28 21:59:04,572 - pytorch_modeler.py - INFO - epoch:57/100, train_losses:8964.084391, val_AUC_hmean:0.679067, val_pAUC_hmean:0.596854, best_flag:False
2021-05-28 21:

100%|██████████| 2/2 [00:00<00:00, 72.11it/s]
2021-05-28 21:59:13,523 - pytorch_modeler.py - INFO - epoch:72/100, train_losses:6810.239583, val_AUC_hmean:0.673616, val_pAUC_hmean:0.596302, best_flag:False
2021-05-28 21:59:13,525 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 24.29it/s]
2021-05-28 21:59:14,022 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 91.68it/s]
2021-05-28 21:59:14,061 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 81.59it/s]
2021-05-28 21:59:14,106 - pytorch_modeler.py - INFO - epoch:73/100, train_losses:6684.906576, val_AUC_hmean:0.670725, val_pAUC_hmean:0.595707, best_flag:False
2021-05-28 21:59:14,107 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 25.03it/s]
2021-05-28 21:59:14,589 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 66.20it/s]
2021-05-28 21:59:14,635 - pytorch_modeler.py - INFO - valid_target
100%|█████████

100%|██████████| 12/12 [00:00<00:00, 25.21it/s]
2021-05-28 21:59:23,394 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 90.43it/s]
2021-05-28 21:59:23,439 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 68.47it/s]
2021-05-28 21:59:23,491 - pytorch_modeler.py - INFO - epoch:89/100, train_losses:4964.872559, val_AUC_hmean:0.677881, val_pAUC_hmean:0.594789, best_flag:False
2021-05-28 21:59:23,492 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 21.45it/s]
2021-05-28 21:59:24,054 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 80.47it/s]
2021-05-28 21:59:24,096 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 79.15it/s]
2021-05-28 21:59:24,142 - pytorch_modeler.py - INFO - epoch:90/100, train_losses:4875.764974, val_AUC_hmean:0.676650, val_pAUC_hmean:0.597881, best_flag:False
2021-05-28 21:59:24,144 - pytorch_modeler.py - INFO - train
100%|█████████

,AUC,pAUC
Source_0,0.786800,0.666316
Source_1,0.923500,0.797368
Source_2,0.806774,0.816587
Target_0,0.452700,0.487368
Target_1,0.675204,0.553169
Target_2,0.596405,0.506364
mean,0.706897,0.637862
h_mean,0.669641,0.611372


elapsed time: 92.781368971 [sec]


In [13]:
#run(machine_types[0])